# **TD5**

# **Partie 2: RNN with Keras**

source: https://stackabuse.com/text-generation-with-python-and-tensorflow-keras/

données: https://www.kaggle.com/ashishsinhaiitr/lord-of-the-rings-text

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import numpy
import sys
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [8]:
#file = open("/content/drive/My Drive/DataSet/texte/LordOfTheRingChap1.txt", encoding="latin-1").read()
file = open("/content/drive/My Drive/DataSet/texte/LordOfTheRingChap1_2_3.txt", encoding="latin-1").read()

In [9]:
import nltk
nltk.download('stopwords')

from nltk.corpus import words
nltk.download('words')
dictionary = set(words.words())

def tokenize_words(input):
    # lowercase everything to standardize it
    input = input.lower()

    # instantiate the tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(input)

    # if the created token isn't in the stop words, make it part of "filtered"
    filtered = filter(lambda token: token not in stopwords.words('english') and token in dictionary , tokens)
    return " ".join(filtered)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [10]:
# preprocess the input data, make tokens
processed_inputs = tokenize_words(file)

In [11]:
chars = sorted(list(set(processed_inputs)))
char_to_num = dict((c, i) for i, c in enumerate(chars))

In [12]:
input_len = len(processed_inputs)
vocab_len = len(chars)
print ("Total number of characters:", input_len)
print ("Total vocab:", vocab_len)

Total number of characters: 80422
Total vocab: 27


In [13]:
seq_length = 100
x_data = []
y_data = []

In [14]:
# loop through inputs, start at the beginning and go until we hit
# the final character we can create a sequence out of
for i in range(0, input_len - seq_length, 1):
    # Define input and output sequences
    # Input is the current character plus desired sequence length
    in_seq = processed_inputs[i:i + seq_length]

    # Out sequence is the initial character plus total sequence length
    out_seq = processed_inputs[i + seq_length]

    # We now convert list of characters to integers based on
    # previously and add the values to our lists
    x_data.append([char_to_num[char] for char in in_seq])
    y_data.append(char_to_num[out_seq])

In [15]:
n_patterns = len(x_data)
print ("Total Patterns:", n_patterns)

Total Patterns: 80322


In [16]:
X = numpy.reshape(x_data, (n_patterns, seq_length, 1))
X = X/float(vocab_len)

In [17]:
y = np_utils.to_categorical(y_data)

In [18]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [20]:
model.fit(X, y, epochs=20, batch_size=256)

Epoch 1/20
314/314 [==============================] - 22s 70ms/step - loss: 2.9187
Epoch 2/20
314/314 [==============================] - 23s 72ms/step - loss: 2.8320
Epoch 3/20
314/314 [==============================] - 23s 72ms/step - loss: 2.5457
Epoch 4/20
314/314 [==============================] - 22s 71ms/step - loss: 2.3924
Epoch 5/20
314/314 [==============================] - 22s 72ms/step - loss: 2.2852
Epoch 6/20
314/314 [==============================] - 23s 72ms/step - loss: 2.1977
Epoch 7/20
314/314 [==============================] - 23s 72ms/step - loss: 2.1224
Epoch 8/20
314/314 [==============================] - 22s 72ms/step - loss: 2.0647
Epoch 9/20
314/314 [==============================] - 22s 71ms/step - loss: 2.0122
Epoch 10/20
314/314 [==============================] - 23s 72ms/step - loss: 1.9632
Epoch 11/20
314/314 [==============================] - 23s 72ms/step - loss: 1.9259
Epoch 12/20
314/314 [==============================] - 22s 71ms/step - loss: 1.8901
E

In [21]:
num_to_char = dict((i, c) for i, c in enumerate(chars))

In [39]:
start = numpy.random.randint(0, 4000)
pattern = x_data[0]
print("Random Seed:")
print("\"", ''.join([num_to_char[value] for value in pattern]), "\"")

Random Seed:
" bilbo bag end would shortly first birthday party special magnificence much talk excitement bilbo ric "


In [41]:
for i in range(50):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(vocab_len)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = num_to_char[index]
    seq_in = [num_to_char[value] for value in pattern]

    sys.stdout.write(result)

    pattern.append(index)
    pattern = pattern[1:len(pattern)]

e said said said said said said said said said sai